<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-03 14:28:01
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.32 C
-------------------
Today PnL: -62.40 K (-0.47%)
Current PnL: -16.52 L (-11.51%)
CY Booked + Current PnL: -12.04 L (-8.39%)
-------------------
Total profit:  4.25 L
Total loss:  -20.76 L
-------------------
Total Booked + Current PnL: 14.94 L (12.82%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.4%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.61 L (60.5%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.42 C (107.83%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.01,X-LC,43.01,93075.0,17953.0,2988.0,0.72,23.90,3.21,27.87,9.0,6.01,0.71,16.73,X40,NTT,CHEMICALS
49,LICI,785.05,983.00,-3.65,H-LC,88.17,236927.0,42235.0,6847.0,-0.73,21.69,2.89,25.21,31.0,6.17,1.81,31.01,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101729.0,23367.0,10661.0,-2.40,29.82,10.48,43.43,70.0,2.19,0.78,36.06,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,11.17,H-SC,91.40,163897.0,26037.0,11719.0,1.82,18.89,7.15,27.39,116.0,2.22,1.26,37.13,X40,ATH,FMCG
85,UNIONBANK,123.87,163.00,-1.84,M-LC,90.32,172653.0,31813.0,12673.0,-0.86,22.59,7.34,31.59,79.0,2.51,1.32,49.98,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,SIS,477.00,528.00,2190.94,M-SC,77.42,62016.0,-19074.0,27746.0,2.14,-23.52,44.74,10.69,201.0,-0.69,0.48,25.68,X40N,NTT,MISC
47,LALPATHLAB,2804.19,3545.00,-32.60,H-SC,41.94,155094.0,864.0,39875.0,1.94,0.56,25.71,26.42,114.0,0.02,1.19,22.05,X40N,NTT,HEALTHCARE
70,SHALBY,261.39,327.00,853.75,M-SC,29.03,129681.0,-51462.0,96924.0,1.89,-28.41,74.74,25.10,206.0,-0.53,0.99,3.79,XY24,NTT,HEALTHCARE
67,SATIN,199.77,274.00,-22.47,H-SC,11.83,153811.0,-42163.0,114989.0,1.86,-21.51,74.76,37.16,122.0,-0.37,1.18,14.86,XY24,NTT,FINANCE
28,GILLETTE,8109.44,10330.69,11.17,H-SC,91.40,163897.0,26037.0,11719.0,1.82,18.89,7.15,27.39,116.0,2.22,1.26,37.13,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,600.83,791.05,-10.68,H-MC,51.61,144380.0,8592.0,34391.0,-3.81,6.33,23.82,31.66,105.0,0.25,1.11,19.06,X40,ATH,INSURANCE
75,SURYODAY,189.78,240.00,60.09,H-SC,67.74,113539.0,-36387.0,76060.0,-2.64,-24.27,66.99,26.46,129.0,-0.48,0.87,45.45,XR,NTT,BANKS
12,BATAINDIA,1550.24,2096.00,-32.17,M-SC,40.86,102032.0,-26638.0,71933.0,-2.52,-20.70,70.50,35.20,197.0,-0.37,0.78,4.10,X40,NTT,FOOTWEAR
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,58.06,101729.0,23367.0,10661.0,-2.40,29.82,10.48,43.43,70.0,2.19,0.78,36.06,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.00,35.36,M-SC,53.76,117423.0,-25056.0,44574.0,-2.25,-17.59,37.96,13.70,221.0,-0.56,0.90,38.59,X40N,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.85,X-LC,49.46,276322.0,14721.0,130894.0,-0.65,5.63,47.37,55.67,1.0,0.11,2.12,13.60,X40,BTT,IT
80,TCS,3794.03,4998.00,-18.41,X-LC,19.35,266331.0,-29603.0,123524.0,-1.02,-10.00,46.38,31.73,2.0,-0.24,2.04,5.66,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.40,X-LC,73.12,182376.0,-17934.0,76726.0,-0.13,-8.95,42.07,29.35,3.0,-0.23,1.40,25.99,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-22.87,X-LC,4.30,153456.0,-47930.0,135548.0,-0.44,-23.80,88.33,43.51,5.0,-0.35,1.18,5.84,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-15.50,X-LC,26.88,265619.0,-4728.0,39365.0,0.99,-1.75,14.82,12.81,6.0,-0.12,2.03,9.93,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,13.28,H-SC,98.92,131042.0,33396.0,14284.0,1.32,34.20,10.90,48.82,134.0,2.34,1.00,45.58,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,90.32,172653.0,31813.0,12673.0,-0.86,22.59,7.34,31.59,79.0,2.51,1.32,49.98,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,58.06,101729.0,23367.0,10661.0,-2.40,29.82,10.48,43.43,70.0,2.19,0.78,36.06,XY24,NTT,MISC
49,LICI,785.05,983.0,-3.65,H-LC,88.17,236927.0,42235.0,6847.0,-0.73,21.69,2.89,25.21,31.0,6.17,1.81,31.01,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.01,X-LC,43.01,93075.0,17953.0,2988.0,0.72,23.90,3.21,27.87,9.0,6.01,0.71,16.73,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,37.37,M-SC,64.52,142690.0,5112.0,89295.0,-1.48,3.72,62.58,68.62,183.0,0.06,1.09,23.70,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,13.28,H-SC,98.92,131042.0,33396.0,14284.0,1.32,34.20,10.90,48.82,134.0,2.34,1.00,45.58,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,11.17,H-SC,91.40,163897.0,26037.0,11719.0,1.82,18.89,7.15,27.39,116.0,2.22,1.26,37.13,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,4.81,H-MC,33.33,99535.0,14434.0,28666.0,-1.21,16.96,28.80,50.65,90.0,0.50,0.76,25.12,X40N,ATH,CABLES
17,CAMS,3643.00,5250.99,4.22,H-SC,70.97,117068.0,15064.0,29958.0,1.01,14.77,25.59,44.14,112.0,0.50,0.90,33.41,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.22,H-SC,70.97,117068.0,15064.0,29958.0,1.01,14.77,25.59,44.14,112.0,0.50,0.90,33.41,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,62.37,142034.0,12882.0,74184.0,-0.35,9.97,52.23,67.42,156.0,0.17,1.09,38.14,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-11.07,M-LC,32.26,152576.0,1631.0,107825.0,-0.64,1.08,70.67,72.51,68.0,0.02,1.17,14.56,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.03,H-SC,56.99,125069.0,1733.0,135237.0,1.54,1.41,108.13,111.05,108.0,0.01,0.96,22.55,AR,ATH,MISC
1,AARTIIND,487.04,919.00,37.28,M-SC,55.91,78857.0,-2479.0,74614.0,0.33,-3.05,94.62,88.69,205.0,-0.03,0.60,32.98,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-23.22,H-SC,65.59,87255.0,-10725.0,32598.0,-1.11,-10.95,37.36,22.32,127.0,-0.33,0.67,26.55,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,58.06,101729.0,23367.0,10661.0,-2.40,29.82,10.48,43.43,70.0,2.19,0.78,36.06,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,53.76,117423.0,-25056.0,44574.0,-2.25,-17.59,37.96,13.70,221.0,-0.56,0.90,38.59,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-380.10,H-SC,100.00,119080.0,-14206.0,98801.0,-0.30,-10.66,82.97,63.47,130.0,-0.14,0.91,99.61,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,13.28,H-SC,98.92,131042.0,33396.0,14284.0,1.32,34.20,10.90,48.82,134.0,2.34,1.00,45.58,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-380.10,H-SC,100.00,119080.0,-14206.0,98801.0,-0.30,-10.66,82.97,63.47,130.0,-0.14,0.91,99.61,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,13.28,H-SC,98.92,131042.0,33396.0,14284.0,1.32,34.20,10.90,48.82,134.0,2.34,1.00,45.58,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6494.44,M-SC,97.85,72339.0,-21471.0,98092.0,-1.41,-22.89,135.60,81.67,214.0,-0.22,0.55,40.09,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,4.42,M-SC,96.77,170828.0,-29388.0,105418.0,-0.74,-14.68,61.71,37.97,195.0,-0.28,1.31,37.16,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-34.59,H-SC,95.70,160280.0,-79135.0,352087.0,-2.21,-33.05,219.67,114.01,113.0,-0.22,1.23,47.00,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,42.01
2,50,70.51


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.43
XY25    14.59
X40     14.43
AR      13.44
X40N    13.24
XR      10.33
X200     1.40
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.22
X-LC    18.73
M-SC    17.36
H-MC     8.44
M-LC     7.65
H-LC     7.62
X-MC     5.55
M-MC     5.50
X-SC     2.74
L-SC     2.26
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.74,-0.30,31.42
IT,10.15,-18.00,82.09
BANKS,9.68,-10.85,61.81
MISC,6.37,-30.07,87.60
FINANCE,5.84,-25.39,84.93
INSURANCE,5.28,3.27,25.71
PAINTS,5.27,-19.75,55.20
ELECTRICAL,5.12,-5.12,50.72
AUTO,4.30,-11.39,60.05


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2869300.0,26
AR,1228792.0,12
X40N,1218665.0,17
XR,1102527.0,14
X40,797095.0,11
XY25,471683.0,10
SR,196715.0,2
X200,76726.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2248235.0,21
M-SC,2047837.0,21
X-LC,863626.0,12
M-MC,550387.0,5
H-MC,525343.0,8
X-SC,351255.0,4
X-MC,307502.0,5
M-LC,305776.0,6
L-SC,301103.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      937871.0      6
M-SC       XY24      755224.0      6
H-SC       AR        547627.0      3
           X40N      499317.0      6
X-LC       X40       481582.0      5
M-SC       X40N      473795.0      6
M-MC       XY24      393340.0      3
M-SC       XR        293640.0      4
H-SC       XR        237417.0      4
M-SC       X40       224508.0      2
H-LC       AR        206509.0      3
M-LC       XY24      183751.0      4
H-MC       XY24      177546.0      2
X-LC       AR        172412.0      2
L-SC       XR        157463.0      2
X-MC       XY24      137214.0      2
X-LC       XY25      132906.0      4
X-SC       X40N      121459.0      2
M-SC       XY25      121337.0      1
X-SC       XR        118179.0      1
H-MC       XR        113819.0      1
X-SC       SR        111617.0      1
M-LC       XR        107825.0      1
X-MC       X40N       95428.0      2
M-SC       AR         94235.0      1
           SR         85098.0      1
M-MC       AR         82863.0      1
L-SC       XY24       81638.0      1
H-MC       XY25       77166.0      1
X-LC       X200       76726.0      1
X-MC       XY25       74860.0      1
M-MC       XR         74184.0      1
H-MC       X40        65002.0      2
           AR         63144.0      1
L-SC       AR         62002.0      1
L-LC       XY25       44367.0      1
H-MC       X40N       28666.0      1
H-SC       X40        26003.0      2
M-LC       XY25       14200.0      1
H-LC       XY25        6847.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
